# Loading Model Requirements

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib notebook

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
import random
import math


import tensorflow as tf
import tensorflow_probability as tfp

from IPython import display
from sklearn import preprocessing
from pickle import dump, load

from matplotlib.ticker import FormatStrFormatter
from IPython.display import SVG

tf.random.set_seed(1234)

tfd = tfp.distributions
tfb = tfp.bijectors
tfk = tf.keras

# Loading Data and Preprocessing

In [2]:
import math
def lonlat2meters(lon, lat):
    semimajoraxis = 6378137.0
    east = lon * 0.017453292519943295
    north = lat * 0.017453292519943295
    t = math.sin(north)
    return semimajoraxis * east, 3189068.5 * math.log((1 + t) / (1 - t))

def meters2lonlat(x, y):
    semimajoraxis = 6378137.0
    lon = x / semimajoraxis / 0.017453292519943295
    t = math.exp(y / 3189068.5)
    lat = math.asin((t - 1) / (t + 1)) / 0.017453292519943295
    return lon, lat

In [3]:
dataset = np.genfromtxt('../processed_nyc_train.csv',delimiter=',', skip_header=1)
dataset = dataset[~np.isnan(dataset).any(axis=1)]

def format_data(dataset, pick_up_scaler=None, drop_off_scaler = None ,save_scaler=True):
    
    pick_up_c, drop_off_c, num_passenger, travel_duration = np.split(dataset, [2, 4, 5], axis = 1)
    
    # Handling of the coordinates
    for i, c in enumerate(pick_up_c):
        lon = pick_up_c[i][0]
        lat = pick_up_c[i][1]
        x, y = lonlat2meters(lon, lat)
        pick_up_c[i][0] = x
        pick_up_c[i][1] = y
    
    if pick_up_scaler is None:
        pick_up_scaler = preprocessing.StandardScaler()
        pick_up_scaler = pick_up_scaler.fit(pick_up_c)
    
    pick_up_c = pick_up_scaler.transform(pick_up_c)
    
    for i, c in enumerate(drop_off_c):
        lon = drop_off_c[i][0]
        lat = drop_off_c[i][1]
        x, y = lonlat2meters(lon, lat)
        drop_off_c[i][0] = x
        drop_off_c[i][1] = y
    
    if drop_off_scaler is None:
        drop_off_scaler = preprocessing.StandardScaler()
        drop_off_scaler = drop_off_scaler.fit(drop_off_c)
    
    drop_off_c = drop_off_scaler.transform(drop_off_c)
    
    
    if save_scaler:
        dump(pick_up_scaler, open('pick_up_scaler.pkl', 'wb'))
        dump(drop_off_scaler, open('drop_off_scaler.pkl', 'wb'))

    final = np.concatenate([pick_up_c, drop_off_c, num_passenger, travel_duration], axis = 1)
    return final

dataset = format_data(dataset)

# Model Definition

In [0]:
import time

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.layers import Layer, Dense, BatchNormalization, ReLU, Conv2D, Reshape
from tensorflow.keras import Model

tfd = tfp.distributions
tfb = tfp.bijectors
tfk = tf.keras

tf.keras.backend.set_floatx('float32')

print('tensorflow: ', tf.__version__)
print('tensorflow-probability: ', tfp.__version__)

from enum import Enum

class Case(Enum):
    sampling = 1
    density_estimation = 2

class NN(Layer):
    """
    Neural Network Architecture for calcualting s and t for Real-NVP
    
    :param input_shape: shape of the data coming in the layer
    :param hidden_units: Python list-like of non-negative integers, specifying the number of units in each hidden layer.
    :param activation: Activation of the hidden units
    """
    def __init__(self, input_shape, n_hidden=[512, 512], activation="relu"):
        super(NN, self).__init__()
        layer_list = []
        for i, hidden in enumerate(n_hidden):
            layer_list.append(Dense(hidden, activation=activation))
        self.layer_list = layer_list
        self.log_s_layer = Dense(input_shape, activation="tanh", name='log_s')
        self.t_layer = Dense(input_shape, name='t')

    def call(self, x):
        y = x
        for layer in self.layer_list:
            y = layer(y)
        log_s = self.log_s_layer(y)
        t = self.t_layer(y)
        return log_s, t

class RealNVP(tfb.Bijector):
    """
    Implementation of a Real-NVP for Denisty Estimation. L. Dinh “Density estimation using Real NVP,” 2016.
    This implementation only works for 1D arrays.
    :param input_shape: shape of the data coming in the layer
    :param hidden_units: Python list-like of non-negative integers, specifying the number of units in each hidden layer.

    """

    def __init__(self, input_shape, n_hidden=[512, 512], forward_min_event_ndims=1, validate_args: bool = False, name="real_nvp"):
        super(RealNVP, self).__init__(
            validate_args=validate_args, forward_min_event_ndims=forward_min_event_ndims, name=name
        )

        assert input_shape % 2 == 0
        input_shape = input_shape // 2
        nn_layer = NN(input_shape, n_hidden)
        x = tf.keras.Input(input_shape)
        log_s, t = nn_layer(x)
        self.nn = Model(x, [log_s, t])
        
    def _bijector_fn(self, x):
        log_s, t = self.nn(x)
        return tfb.affine_scalar.AffineScalar(shift=t, log_scale=log_s)

    def _forward(self, x):
        x_a, x_b = tf.split(x, 2, axis=-1)
        y_b = x_b
        y_a = self._bijector_fn(x_b).forward(x_a)
        y = tf.concat([y_a, y_b], axis=-1)
        return y

    def _inverse(self, y):
        y_a, y_b = tf.split(y, 2, axis=-1)
        x_b = y_b
        x_a = self._bijector_fn(y_b).inverse(y_a)
        x = tf.concat([x_a, x_b], axis=-1)
        return x

    def _forward_log_det_jacobian(self, x):
        x_a, x_b = tf.split(x, 2, axis=-1)
        return self._bijector_fn(x_b).forward_log_det_jacobian(x_a, event_ndims=1)
    
    def _inverse_log_det_jacobian(self, y):
        y_a, y_b = tf.split(y, 2, axis=-1)
        return self._bijector_fn(y_b).inverse_log_det_jacobian(y_a, event_ndims=1)

tensorflow:  2.1.0
tensorflow-probability:  0.9.0


In [0]:
batch_size = 100
layers = 12
# base_lr = 1e-3
# end_lr = 1e-4
# max_epochs = int(100)
shape = [128, 128]
exp_number = 1
input_shape = 6
permutation = tf.cast(np.concatenate((np.arange(input_shape/2,input_shape),np.arange(0,input_shape/2))), tf.int32)
event_shape = [6]

In [0]:
base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros(shape=input_shape, dtype=tf.float32))

bijectors = []
alpha = 1e-3

for i in range(layers):
    bijectors.append(tfb.BatchNormalization())
    bijectors.append(RealNVP(input_shape=input_shape, n_hidden=shape))
    bijectors.append(tfp.bijectors.Permute(permutation))


bijector = tfb.Chain(bijectors=list(reversed(bijectors)), name='chain_of_real_nvp')

flow = tfd.TransformedDistribution(
    distribution=base_dist,
    bijector=bijector
)

# number of trainable variables
n_trainable_variables = len(flow.trainable_variables)

# Model Training Parameters

In [0]:
base_lr = 1e-3
end_lr = 1e-4
max_epochs = int(5e3)  # maximum number of epochs of the training
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(base_lr, max_epochs, end_lr, power=0.5)

In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn) 

x_ = tf.keras.layers.Input(shape=event_shape, dtype=tf.float32)
log_prob_ = flow.log_prob(x_)
model = tf.keras.Model(x_, log_prob_)

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam() ,
              loss=lambda _, log_prob: -log_prob)

# Setting Up Checkpoint ( Will Save Model After Every Epoch)

In [0]:
batch_size = 100
weight_file = './checkpoint/cp.h5'

In [0]:
for i in range(100): # set to the desired epoch
  model.load_weights(weight_file)
  model.fit(x=dataset,
            y=np.zeros((dataset.shape[0], 0), dtype=np.float32),
            batch_size=batch_size,
            epochs=1,
            steps_per_epoch=dataset.shape[0] // batch_size,
            shuffle=True,
            verbose=True)
  model.save_weights(weight_file)

Train on 1458643 samples
1396800/1458643 [===========================>..] - ETA: 34s - loss: -251.0287

In [0]:
model.load_weights(weight_file)
model.save_weights("RNVP_12_128_128.h5")

# Generating Samples

In [0]:
output = flow.sample(100000).numpy()

In [0]:
def reconstruct(predictions):
    
    # split the output first
    pick_up_c, drop_off_c, num_passenger, travel_duration = np.split(dataset, [2, 4, 5], axis = 1)
    
    # recover scaler
    pick_up_scaler = load(open('pick_up_scaler.pkl', 'rb'))
    drop_off_scaler = load(open('drop_off_scaler.pkl', 'rb'))
    pick_up_c = pick_up_scaler.inverse_transform(pick_up_c)
    drop_off_c = drop_off_scaler.inverse_transform(drop_off_c)

    for i, c in enumerate(pick_up_c):
      x = pick_up_c[i][0]
      y = pick_up_c[i][1]
      lon, lat = meters2lonlat(x, y)
      pick_up_c[i][0] = lon
      pick_up_c[i][1] = lat
    
    for i, c in enumerate(pick_up_c):
      x = drop_off_c[i][0]
      y = drop_off_c[i][1]
      lon, lat = meters2lonlat(x, y)
      drop_off_c[i][0] = lon
      drop_off_c[i][1] = lat
    
    return np.concatenate([pick_up_c, drop_off_c, num_passenger, travel_duration], axis = 1)

In [0]:
samples = reconstruct(output)

# Saving Generated Samples

In [0]:
file_name = '100000_samples_RNVP_12_128_128' + '.csv'
np.savetxt(file_name, samples, delimiter = ',', header='pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, passenger_count, trip_duration' )